In [1]:
import os

# Your desired directory path
desired_path = "c:/Projects/taxi_demand_predictory/"

# Change the current working directory to the specified path
os.chdir(desired_path)

# Verify by printing the current working directory
print("Current Working Directory: ", os.getcwd())


Current Working Directory:  c:\Projects\taxi_demand_predictory


In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-29,1,0.0
1,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,0.0,2.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,2023-01-30,1,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,2.0,0.0,1.0,0.0,0.0,0.0,2023-01-31,1,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-02-01,1,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2023-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40540,14.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,...,4.0,8.0,5.0,4.0,11.0,3.0,8.0,2023-06-26,265,5.0
40541,2.0,1.0,2.0,4.0,1.0,0.0,2.0,2.0,0.0,2.0,...,8.0,3.0,9.0,5.0,3.0,5.0,7.0,2023-06-27,265,3.0
40542,2.0,3.0,2.0,0.0,1.0,1.0,0.0,2.0,0.0,5.0,...,8.0,6.0,2.0,7.0,7.0,8.0,3.0,2023-06-28,265,4.0
40543,0.0,5.0,0.0,2.0,0.0,1.0,1.0,3.0,5.0,7.0,...,15.0,11.0,3.0,2.0,1.0,8.0,7.0,2023-06-29,265,9.0


In [10]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2023, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(7950, 674)
y_test.shape=(7950,)


In [11]:
import xgboost as xgb

In [12]:
# use only past rides data
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]

In [13]:
model = xgb.XGBRegressor()
model.fit(X_train_only_numeric, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [14]:

X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)
predictions

array([0.11106471, 0.13039412, 0.15733002, ..., 2.5393083 , 4.6900125 ,
       4.920179  ], dtype=float32)

In [15]:
from sklearn.metrics import mean_absolute_error, r2_score
test_mae = mean_absolute_error(y_test, predictions)
test_r2 = r2_score(y_test, predictions)
print(f'{test_mae=:.4f}')
print(f'{test_r2=:.4f}')

test_mae=2.7694
test_r2=0.9338
